In [9]:
from os import listdir
import string
from tqdm import tqdm
import re
import pandas as pd
import numpy as np

import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
import tensorflow_datasets as tfds
import time
import logging
logging.basicConfig(level=logging.ERROR)


## loading best model


In [10]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

In [12]:

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')


val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  16449536  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  18881280  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  25176064  
Total params: 60,506,880
Trainable params: 60,506,880
Non-trainable params: 0
_________________________________________________________________


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
%cd /content/gdrive/My\ Drive/

/content/gdrive/My Drive


### loading best_model weights

In [16]:
model.load_weights('t5model.h5')

In [ ]:
BATCH_SIZE = 64
SHUFFEL_SIZE = 1024
learning_rate = 3e-5

### Function1

In [17]:
def fun1(path):
    
    directory = path
    
    
    def loading_articles(file_name):
        file = open(file_name, encoding='utf-8')
        text = file.read()
        file.close()
        return text
    
    def split_story(doc):
        index = doc.find('@highlight')
        story, highlights = doc[:index], doc[index:].split('@highlight')
        highlights = [h.strip() for h in highlights if len(h) > 0]
        return story, highlights


    
    stories = list()
    for name in (listdir(directory)):
        filename = directory + '/' + name
        doc = loading_articles(filename)
        story, highlights = split_story(doc)
        stories.append({'story':story, 'highlights':highlights})
        
   
    CNN = pd.DataFrame.from_dict(cnn_stories)
    CNN.columns = ['article','summary']
    CNN.to_csv('CNN_with_summary.csv',index=False)


    def decontracted(phrase):

        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

    
    article_text=[]
    summary_text=[]
    for i in CNN.article.values:
        tt=re.sub(r'\n',' ', i)
        tt=re.sub(r'>',' ', tt)
        tt=re.sub(r'<',' ', tt)
        tt=re.sub(r'(CNN)',' ', tt)
        tt=re.sub(r'LRB',' ', tt)
        tt=re.sub(r'RRB',' ', tt)
        tt = re.sub(r'[" "]+', " ", tt)
        tt=re.sub(r'-- ',' ', tt)
        tt=re.sub(r"([?!¿])", r" \1 ", tt)
        tt=re.sub(r'-',' ', tt)
        tt=tt.replace('/',' ')
        tt=re.sub(r'\s+', ' ', tt)
        tt=decontracted(tt)
        tt = re.sub('[^A-Za-z0-9.,]+', ' ', tt)
        tt = tt.lower()
        article_text.append(tt)
    article_text=np.array(article_text)
    article_text=article_text.reshape(-1,1)
    data_article=pd.DataFrame(article_text,columns=['Article'])

   
    for i in CNN.summary.values:
        tt=re.sub(r'\n',' ', i)
        tt=re.sub(r'>',' ', tt)
        tt=re.sub(r'<',' ', tt)
        tt=re.sub(r'-',' ', tt)
        tt = re.sub(r'[" "]+', " ", tt)
        tt=re.sub(r'-- ',' ', tt)
        tt=tt.replace('/',' ')
        tt=re.sub(r'\s+', ' ', tt)
        tt=decontracted(tt)
        tt = re.sub('[^A-Za-z0-9.]+', ' ', tt)
        tt = tt.lower()
        summary_text.append(tt)
    summary_text=np.array(summary_text)
    summary_text=summary_text.reshape(-1,1)
    data_summ=pd.DataFrame(summary_text,columns=['Summary'])
    data_cleaned=data_article.join(data_summ)
    
    
    def normalize_text(text):
    
        text = tf.strings.lower(text)
        text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
        return text.numpy().decode('UTF-8')

    def tokenize_articles(text):
        text = normalize_text(text)
        ids = tokenizer.encode_plus((model.config.prefix + text), return_tensors="tf", max_length=350, truncation='longest_first') 

        return tf.squeeze(ids['input_ids']), tf.squeeze(ids['attention_mask'])
        
    def tokenize_highlights(text):
        text = normalize_text(text)
        ids = tokenizer.encode(text, return_tensors="tf", max_length=50)
        return tf.squeeze(ids)


    def map_func(x, y):
        article_ids, attention_mask = tf.py_function(tokenize_articles, inp=[x], Tout=(tf.int32, tf.int32))
        highlights_ids = tf.py_function(tokenize_highlights, inp=[y], Tout=tf.int32)

        return article_ids, attention_mask, highlights_ids

    
    tfds = tf.data.Dataset.from_tensor_slices((data_cleaned.Article.values, data_cleaned.Summary.values))
 

    ds = tfds.map(map_func)\
    .shuffle(1024)\
    .padded_batch(64, padded_shapes=([350],[350],[50]))\
    .prefetch(tf.data.experimental.AUTOTUNE)





    predictions = []
    for i, (input_ids, input_mask, y) in (enumerate(ds)):
        summaries = model.generate(input_ids=input_ids,max_length=45 ,attention_mask=input_mask)
        pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
        predictions.append(pred)

    return predictions
    


### Function_2

### fun2(X,labels)

### in our case summaries are present in raw text so didnt give label arg to fun2  

In [44]:
!pip install sumeval

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 17.1MB 257kB/s 
  Created wheel for sumeval: filename=sumeval-0.2.2-cp36-none-any.whl size=54535 sha256=b9a7ee93e54e113f522142a92a0f96e80570d88960d3eec31fb9cad5a26e0b47
  Stored in directory: /root/.cache/pip/wheels/7b/6f/57/19ceecab21445c88f3c565735fa1887b4cd18d340c972eb445
Successfully built sumeval


In [45]:
from sumeval.metrics.rouge import RougeCalculator


In [46]:
def fun2(path):
    
    directory = path
    
    
    def loading_articles(file_name):
        file = open(file_name, encoding='utf-8')
        text = file.read()
        file.close()
        return text
    
    def split_story(doc):
        index = doc.find('@highlight')
        story, highlights = doc[:index], doc[index:].split('@highlight')
        highlights = [h.strip() for h in highlights if len(h) > 0]
        return story, highlights


    

    stories = list()
    for name in (listdir(directory)):
        filename = directory + '/' + name
        doc = loading_articles(filename)
        story, highlights = split_story(doc)
        stories.append({'story':story, 'highlights':highlights})
        
   
    CNN = pd.DataFrame.from_dict(stories)
    CNN.columns = ['article','summary']



    def decontracted(phrase):

        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

    
    article_text=[]
    summary_text=[]
    for i in CNN.article.values:
        tt=re.sub(r'\n',' ', i)
        tt=re.sub(r'>',' ', tt)
        tt=re.sub(r'<',' ', tt)
        tt=re.sub(r'(CNN)',' ', tt)
        tt=re.sub(r'LRB',' ', tt)
        tt=re.sub(r'RRB',' ', tt)
        tt = re.sub(r'[" "]+', " ", tt)
        tt=re.sub(r'-- ',' ', tt)
        tt=re.sub(r"([?!¿])", r" \1 ", tt)
        tt=re.sub(r'-',' ', tt)
        tt=tt.replace('/',' ')
        tt=re.sub(r'\s+', ' ', tt)
        tt=decontracted(tt)
        tt = re.sub('[^A-Za-z0-9.,]+', ' ', tt)
        tt = tt.lower()
        article_text.append(tt)
    article_text=np.array(article_text)
    article_text=article_text.reshape(-1,1)
    
    data_article=pd.DataFrame(article_text,columns=['Article'])

   
    for i in CNN.summary.values:
        tt=re.sub(r'\n',' ', i)
        tt=re.sub(r'>',' ', tt)
        tt=re.sub(r'<',' ', tt)
        tt=re.sub(r'-',' ', tt)
        tt = re.sub(r'[" "]+', " ", tt)
        tt=re.sub(r'-- ',' ', tt)
        tt=tt.replace('/',' ')
        tt=re.sub(r'\s+', ' ', tt)
        tt=decontracted(tt)
        tt = re.sub('[^A-Za-z0-9.]+', ' ', tt)
        tt = tt.lower()
        summary_text.append(tt)
    summary_text=np.array(summary_text)
    summary_text=summary_text.reshape(-1,1)
    
    data_summ=pd.DataFrame(summary_text,columns=['Summary'])
    data_cleaned=data_article.join(data_summ)
    
    
    def normalize_text(text):
    
        text = tf.strings.lower(text)
        text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
        return text.numpy().decode('UTF-8')

    def tokenize_articles(text):
        text = normalize_text(text)
        ids = tokenizer.encode_plus((model.config.prefix + text), return_tensors="tf", max_length=350, truncation='longest_first') 

        return tf.squeeze(ids['input_ids']), tf.squeeze(ids['attention_mask'])
        
    def tokenize_highlights(text):
        text = normalize_text(text)
        ids = tokenizer.encode(text, return_tensors="tf", max_length=50)
        return tf.squeeze(ids)


    def map_func(x, y):
        article_ids, attention_mask = tf.py_function(tokenize_articles, inp=[x], Tout=(tf.int32, tf.int32))
        highlights_ids = tf.py_function(tokenize_highlights, inp=[y], Tout=tf.int32)

        return article_ids, attention_mask, highlights_ids

    
    tfds = tf.data.Dataset.from_tensor_slices((data_cleaned.Article.values, data_cleaned.Summary.values))
 

    ds = tfds.map(map_func)\
    .shuffle(1024)\
    .padded_batch(64, padded_shapes=([350],[350],[50]))\
    .prefetch(tf.data.experimental.AUTOTUNE)


    predictions = []
    reference=[]
    for i, (input_ids, input_mask, y) in (enumerate(ds)):
        summaries = model.generate(input_ids=input_ids,max_length=45 ,attention_mask=input_mask)
        pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
        real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
        predictions.append(pred)
        reference.append(real)


    rouge = RougeCalculator(stopwords=True, lang="en")
    Rouge_1 = []
    Rouge_2 = []
    Rouge_l = []

    for i in range(len(predictions)):

        rouge_1 = rouge.rouge_n(
        summary=predictions[i],
        references=reference[i],
        n=1)
        Rouge_1.append(rouge_1)
 
 
 ## rogue_2

        rouge_2 = rouge.rouge_n(
        summary=predictions[i],
        references=reference[i],
        n=2)
        Rouge_2.append(rouge_2)
 ## rogue_l

        rouge_l = rouge.rouge_l(
        summary=predictions[i],
        references=reference[i])
        Rouge_l.append(rouge_l)




    return print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-L: {}".format(np.round(sum(Rouge_1)/len(Rouge_1),3),
 np.round(sum(Rouge_2)/len(Rouge_2),3),
 np.round(sum(Rouge_l)/len(Rouge_l),3)
))
    

In [47]:
path='cnn_data'
Metric_score=fun2(path)

ROUGE-1: 0.401, ROUGE-2: 0.173, ROUGE-L: 0.367
